## Tutorial 06: 3D Radon-Cumulative Distribution Transform Nearest Subspace (3D-RCDT-NS) Classifier

This tutorial will demonstrate how to use the 3D-RCDT-NS classifier in the *PyTransKit* package.

# Class:: RCDT_NS_3D

Functions:
--------------
1. Constructor function:
        rcdt_ns_obj = RCDT_NS(num_classes, thetas, rm_edge)
        
        Inputs:
        ----------------
        num_classes : integer value
            totale number of classes in the dataset.
        Npoints : scalar; number of radon projections
        use_gpu : boolean; IF TRUE, use GPU to calculate 3D RCDT
        rm_edge : boolean 
            IF TRUE, the first and last points of RCDTs will be removed.
            
        Outputs:
        ----------------
        rcdt_ns_obj : class object
            Instance of the class RCDT_NS.
            
2. Fit function:
        rcdt_ns_obj.fit(Xtrain, Ytrain, no_deform_model)
        
        Inputs:
        ----------------
        Xtrain : 4d array, shape (n_samples, L, L, L)
            3D Image data for training. L is the dimension along X,Y, and Z axes.
        Ytrain : 1d array, shape (n_samples,)
            Labels of the training images.
        no_deform_model : boolean
            IF TRUE, no deformation model will be added
  
3. Predict function:
        preds = rcdt_ns_obj.predict(Xtest, use_gpu)
        
        Inputs:
        ----------------
        Xtest : 4d array, shape (n_samples, L, L, L)
            3D Image data for testing. L is the dimension along X,Y, and Z axes.
        use_gpu : boolean 
            IF TRUE, use gpu for calculations.
            
        Outputs:
        ----------------
        preds : 1d array, shape (n_samples,)
           Predicted labels for test samples.
    

# Example
The following example will demonstrate how to:
* create and initialize an instance of the class 3D-RCDT_NS
* train the model with training images
* apply the model to predict calss labels of the test images
In this example we have used MNIST dataset stored in the *data* folder

### Import some python libraries

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
from pathlib import Path
import sys
sys.path.append('../')
from pytranskit.classification.utils import *

use_gpu = False

### Import 3D-RCDT-NS class from *PyTransKit* package

In [2]:
from pytranskit.classification.rcdt_ns_3d import RCDT_NS_3D

### Load dataset
For loading data we have used *load_data_3D* function from the *pytranskit/classifier/utils.py* script. It takes name and directory of the dataset, and total number of classes as input. Returns both train and test images in two separate 4d arrays of shape (n_samples, n_rows, n_columns, n_columns), and corresponding class labels. User can use there own implementation to load data, just need to make sure that the output arrays are consistent.<br>
<br>In this example, we have used a synthetic 3D dataset with two classes: class 0 contains one Gaussian blob in each image, class 1 contains two Gaussian blobs in each image.<br>
<br>Note: The 3D RCDT implemented in PyTransKit, 3D images need to be equal shape along all three directions, i.e. *n_rows=n_columns=n_columns=L*. Therefore, if the original image does not have equal length in all axes, users need to zero pad to make all the dimensions equal.

In [3]:
datadir = './data'
dataset = 'synthetic_3D'
num_classes = 2          # total number of classes in the dataset
(x_train, y_train), (x_test, y_test) = load_data_3D(dataset, num_classes, datadir)  # load_data function from utils.py

loaded from cache file data: x_train (100, 32, 32, 32) x_test (100, 32, 32, 32)


In this example we have used 32 randomly chosen samples per class to train the model. We have used another function *take_train_samples* function from *utils.py* script for this. User can use their own script.

In [4]:
n_samples_perclass = 32  # total number of training samples per class used in this example
x_train_sub, y_train_sub = take_train_samples(x_train, y_train, n_samples_perclass, 
                                              num_classes, repeat=0) # function from utils.py

### Create an instance of 3D-RCDT-NS class

In [5]:
Npoints = 500    # choose number projections 3D Radon transform
rcdt_ns_obj = RCDT_NS_3D(num_classes, Npoints, use_gpu=False, rm_edge=True)

### Training phase
This function takes the train samples and labels as input, and stores the basis vectors for corresponding classes in a *private* variable. This variable will be used in the *predict* function in the test phase

In [6]:
rcdt_ns_obj.fit(x_train_sub, y_train_sub)


Calculating RCDTs for training images ...
Generating basis vectors for each class ...


### Testing phase
*predict* function takes the train samples as input and returns the predicted class labels

In [7]:
preds = rcdt_ns_obj.predict(x_test, use_gpu)


Calculating RCDTs for testing images ...
Finding nearest subspace for each test sample ...


In [8]:
print('\nTest accuracy: {}%'.format(100*accuracy_score(y_test, preds)))


Test accuracy: 98.0%
